# D-Hydro Boezemmodel V4 - Modelbouw

Met deze notebook bouw je een boezemmodel voor Nooderzijlvest. 

De notebook is opgezet door Daniel Tollenaar (D2Hydro), Vincent de Looij (Waterschap Noorderzijlvest) en Siebe Bosch (HydroConsult). Het Notebook is gebaseerd op een  <a href="https://github.com/openearth/delft3dfmpy/blob/master/notebooks/Usage_introduction_FM_only.ipynb">voorbeeldnotebook</a> uit D-HyDAMO, opgezet door HKV Lijn in Water: 


## Installatie
Zorg dat je beschikt over een werkende Python installatie en <a href="https://github.com/d2hydro/delft3dfmpy#installation">D-HYDAMO omgeving</a>. 

Het resulterende model moet geimporteerd kunnen worden in D-HYDRO versie 0.9.7.52006 of hoger.

Download de bestanden van <a href="https://drive.google.com/drive/folders/1BXNEwlEDCtShDj5KVuHQcukup-QLTuzR">Google Drive</a> en zet deze in de folder .\data

De verwijzing naar bestanden is geschreven in het Nederlands. De rest van de code is geschreven in het Engels. Elk code-blok is voorzien van uitleg boven het desbetreffende blok, geschreven in het Nederlands.

<b>let op:</b> deze notebook werkt alleen in combinatie met een config.py. In config.py dient een variabele dimr_path te zijn opgegeven, afhankelijk van de D-Hydro installatie. Bijvoorbeeld:

dimr_path = r"c:\Program Files (x86)\Deltares\D-HYDRO Suite 1D2D (Beta) (0.9.7.52006)\plugins\DeltaShell.Dimr\kernels\x64\dimr\scripts\run_dimr.bat"

## Controle bestanden

Alle bestanden die gebruikt worden in deze tutorial staan in het code-blok hieronder. Wanneer je dit codeblok uitvoert wordt de aanwezigheid van deze bestanden gecontroleerd.



In [1]:
from pathlib import Path
from config import dimr_path
import pandas as pd
import numpy as np
import rasterio
from rasterstats import zonal_stats

data_path = Path(r".\data").absolute().resolve()
beheerregister = data_path.joinpath("beheerregister")
excelbestanden = data_path.joinpath("xlsx")
bathymetrie = data_path.joinpath("bathymetrie")

modelbestanden = {"randvoorwaarden":"randvoorwaarden.xlsx",
                  "uitgesloten_profielen":"uitgesloten_profielen.xlsx"}

invoerbestanden = {
    "modelgebied": "Bemalings_en_lozingsgebieden.shp",
    "branches": "Afaanvoervakken_DHYDRO_v6.shp",
    "profielpunten": "profielpunten_3feb2021_actueel.shp",
    "bruggen": "Bruggen_(doorstroomopeningen).shp",
    "duikers": "Duikers_v2.shp",
    "sluizen": "sluizen.shp",
    "sifons": "sifons.shp",
    "stuwen": "Stuwen_v2_zonderdrempels.shp",
    "inlaten": "inlaten.shp",
    "gemalen": "Gemalen_v3.shp",
    "peilgebieden": "Peilgebieden.shp"
}

bathymetriebestanden = {
    "hoogte_raster": "AHN3_Lauwersmeer_gevuld.tif",
    "zones": "Polygonen_Lauwersmeer.shp"
}

vorm_mapping = dict(rond=1,
                    driehoekig=1,
                    rechthoekig=3,
                    eivormig=1,
                    ellips=1,
                    Paraboolvormig=1,
                    trapeziumvormig=1,
                    heul=1,
                    muil=3,
                    langwerpig=1,
                    scherp=1,
                    onbekend=99,
                    overig=99)

ruwheid_mapping = {"A1": 0.03,
                   "A2": 0.03,
                   "A2 Boot": 0.03,
                   "A3": 0.03,
                   "B1": 0.02,
                   "B2": 0.022,
                   "C1": 0.02,
                   "C2b": 0.022,
                   "DERDEN": 0.03,
                   "GEEN": 0.07}

richting_mapping = {"schutsluis naar een zijde": "none",
                    "overig": "both",
                    "schutsluis naar twee zijden": "none",
                    "uitwateringssluis/spuisluis": "positive"}

for key, item in invoerbestanden.items():
    if not beheerregister.joinpath(item).exists():
        print(f"bestand voor {key} bestaat niet: {beheerregister.joinpath(item)}")
        
for key, item in bathymetriebestanden.items():
    if not bathymetrie.joinpath(item).exists():
        print(f"bestand voor {key} bestaat niet: {bathymetrie.joinpath(item)}")
        
for key, item in modelbestanden.items():
    if not excelbestanden.joinpath(item).exists():
        print(f"bestand voor {key} bestaat niet: {excelbestanden.joinpath(item)}")

rekenpunt_afstand = 250.0


## Inlezen beheerregister in HyDAMO

### Aanmaken HyDAMO object

Alle benodigde modules worden geimporteerd en het HYDAMO object wordt aangemaakt bij het uitvoeren van onderstaand code-blok

In [2]:
from delft3dfmpy import DFlowFMModel, HyDAMO, Rectangular, DFlowFMWriter
from delft3dfmpy import DFlowRRModel, DFlowRRWriter
from delft3dfmpy.datamodels.common import ExtendedGeoDataFrame
import hydrotools
import geopandas as gpd
import pandas as pd
from shapely.geometry import LineString

hydamo = HyDAMO(extent_file=str(beheerregister.joinpath(invoerbestanden["modelgebied"])))

### hulp-variabelen

#### snap_to_branches:
De optie om bij het inlezen te snappen naar branches en alles dat niet op boezemmodel_v4 ligt weg te gooien. Wordt nu alleen gebruikt bij duikers.

ToDo:
- We gebruiken de delft3dfmpy functie "find_nearest_branch" om te snappen naar de branches. Het lijkt erop dat deze, bij de afstand tussen lijn-objecten (duikers, hevels) niet het middenpunt van de lijn pakt maar de lijn zelf. Gevolg: kruisende lijnen (sifons) worden gesnapt naar de kruisende tak. Beter het middenpunt snappen en niet de lijn zelf.

#### kwk_groups_df:
De kunstwerkgroepen waarmee we compound-structures genereren

In [3]:
branches_gdf = gpd.read_file(beheerregister.joinpath(invoerbestanden["branches"]))
branches_gdf.set_index("OVKIDENT", inplace=True)
branches_gdf.columns = branches_gdf.columns.str.lower()

snap_to_branches = dict(branches=branches_gdf,
                        distance=1,
                        attribute_filter={"hydromodel": "Boezemmodel_v4"}) 

kwk_groups_df = pd.read_csv(excelbestanden.joinpath("kunstwerkgroepen.csv"), index_col="code")

### Toevoegen waterlopen

De waterlopen (branches) worden toegevoegd met volgend code-blok:
* de branches worden toegekend aan het hydamo-object
* een HyDAMO ruwheidscode (2 = manning) wordt toegekend
* De ruwheidswaarde wordt bepaalt aan de hand van de werkcode
* De branches worden versimpeld aan de hand van het bestand verplaats_eindpunten.shp
* de eindpunten van de branches worden gesnapped binnen een tolerantie van 1m. De coordinaten worden ook afgerond op 1m.

In [4]:
gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["branches"]),
                           "branches",
                           attribute_filter={"hydromodel": "Boezemmodel_v4"},
                           keep_columns=["HydroModel",
                                         "bodemhoogtebovenstrooms",
                                         "bodembreedte",
                                         "taludhellinglinkerzijde",
                                         "taludhellingrechterzijde",
                                         "IWS_W_WATB",
                                         "WERKCODE"],
                           column_mapping={"OVKIDENT": "code",
                                           "IWS_AVVHOB": "bodemhoogtebovenstrooms",
                                           "AVVBODDR": "bodembreedte",
                                           "AVVTALUL": "taludhellinglinkerzijde",
                                           "AVVTALUR": "taludhellingrechterzijde"}
                          )

gdf.loc[:, "ruwheidstypecode"] = 2
gdf.loc[:, "ruwheidswaarde"] = gdf.apply((lambda x: ruwheid_mapping[x["werkcode"]]), axis=1)

# verplaatsen eindpunten
move_lines_gdf = gpd.read_file(data_path.joinpath("shp","verplaats_eind_nodes.shp"))
gdf = hydrotools.move_end_nodes(gdf, move_lines_gdf, threshold=1)

hydamo.branches.set_data(gdf, index_col="code", check_columns=True, check_geotype=True)

hydamo.branches = hydrotools.snap_ends(hydamo.branches, tolerance=1, digits=1)

hydamo.branches = hydamo.branches.loc[~hydamo.branches.index.isin(["OAF016902",
                                                                   "OAF016901"])]

C:\Anaconda3\envs\delft3dfmpy\lib\site-packages\pandas\core\frame.py:4312: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


### Aanmaken principeprofielen
Als terugvaloptie voor branches zonder profielen schrijven we principe profielen weg die we later in deze notebook gebruiken.

In [5]:
principe_profielen_df = hydrotools.get_trapeziums(gdf,
                                                  "code",
                                                  "bodembreedte",
                                                  "bodemhoogtebovenstrooms",
                                                  "iws_w_watb",
                                                  "taludhellinglinkerzijde",
                                                  "taludhellingrechterzijde",
                                                  "ruwheidstypecode",
                                                  "ruwheidswaarde")

principe_profielen_df.to_csv(excelbestanden.joinpath("principe_profielen.csv"))

### Toevoegen yz-profielen
De yz-profielen worden ingeladen:
* De profielpunten worden geopend
* de punten worden geordend en geconverteerd naar polylinen met een xyz coordinaten
* lijnen langer dan 500m worden weggegooid, omdat er soms kademuren PRO_TYPE = PRO hebben gekregen
* lijnen worden toegekend aan het HyDAMO object
* lijnen die niet snappen met de branches, worden weggegooid
* lijnen krijgen de ruwheidswaarde mee van de onderliggende tak (branch)

In [6]:
gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["profielpunten"]),
                           "crosssections",
                           attribute_filter={"osmomsch_1": ["Z1"],
                                             "PRO_TYPE": "PRO"},
                           column_mapping={"PROIDENT": "code",
                                           "IWS_VOLGNR": "order",
                                           "OSMOMSCH_1": "category",
                                           "iws_hoogte": "z"},
                           z_coord=True
                           )

grouper = gdf.groupby("code")
profiles = dict()

for code, prof_gdf in grouper:
    prof_gdf = prof_gdf.sort_values("order")  #Staan ook punten met zelfde order
    first_point = prof_gdf.iloc[0]["geometry"]
    cum_dist = -1
    line = []
    for idx, (_, row) in enumerate(prof_gdf.iterrows()):
        geom = row["geometry"]
        distance = geom.distance(first_point)
        if distance > cum_dist:
            cum_dist = distance
            line += [(geom.x, geom.y, row["z"])]

    profiles[code] = [code, LineString(line)]

profiles_gdf = gpd.GeoDataFrame.from_dict(profiles,
                                          orient="index",
                                          columns=["code",
                                                   "geometry"]
                                          )

profiles_gdf["ruwheidstypecode"] = 2

profiles_gdf["codegerelateerdobject"] = None

profiles_gdf = profiles_gdf.loc[profiles_gdf["geometry"].length < 500]

exclude_xlsx = excelbestanden.joinpath(modelbestanden["uitgesloten_profielen"])
exclude_list = pd.read_excel(exclude_xlsx, header=None)[0].to_list()
profiles_gdf = profiles_gdf.loc[~profiles_gdf.index.isin(exclude_list)]

hydamo.crosssections.set_data(profiles_gdf,
                              index_col="code",
                              check_columns=False,
                              check_geotype=True)

hydamo.crosssections.snap_to_branch(hydamo.branches, snap_method="intersecting")
hydamo.crosssections.dropna(axis=0, inplace=True, subset=["branch_offset"])
hydamo.crosssections["ruwheidswaarde"] = hydamo.crosssections.apply(
    (lambda x: hydamo.branches.loc[x["branch_id"]]["ruwheidswaarde"]), axis=1)


### Toevoegen geparameteriseerde profielen

Per brug moet er een profiel worden toegevoegd

In [7]:
gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["bruggen"]),
                           "parametrised_profiles",
                           column_mapping={"bodemh_bov": "bodemhoogtebovenstrooms",
                                           "bodemh_ben": "bodemhoogtebenedenstrooms",
                                           "doorstroom": "bodembreedte",
                                           "hoogte_ond": "hoogteinsteeklinkerzijde",
                                           })

grouper = gdf.groupby("code")
data = {item: [] for item in ["code",
                              "bodemhoogtebovenstrooms",
                              "bodemhoogtebenedenstrooms",
                              "bodembreedte",
                              "hoogteinsteeklinkerzijde",
                              "geometry",
                              "codegerelateerdobject"]}

for code, df in grouper:
    data["code"] += [f"PRO_{code}"]
    data["codegerelateerdobject"] += [code]
    for item in ["bodemhoogtebovenstrooms",
                 "bodemhoogtebenedenstrooms",
                 "hoogteinsteeklinkerzijde",
                 "geometry"]:
        data[item] += [df.iloc[0][item]]
    data["bodembreedte"] += [df['bodembreedte'].sum()]

gdf = gpd.GeoDataFrame(data)
gdf["geometry"] = gdf.apply((lambda x: LineString([[x["geometry"].x-0.1,
                                                    x["geometry"].y-0.1],
                                                  [x["geometry"].x+0.1,
                                                   x["geometry"].y+0.1]])),
                            axis=1)

gdf["hoogteinsteekrechterzijde"] = gdf["hoogteinsteeklinkerzijde"]
gdf["ruwheidswaarde"] = 15
gdf["ruwheidstypecode"] = 4
gdf["taludhellinglinkerzijde"] = 1
gdf["taludhellingrechterzijde"] = 1

#alle geparmeteriseerde profielen met bodemhoogte 0m (NAP) weggooien
gdf = gdf.loc[gdf["bodemhoogtebovenstrooms"] != 0]
hydamo.parametrised_profiles.set_data(gdf,
                                      index_col="code",
                                      check_columns=True,
                                      check_geotype=False)

### Toevoegen gemalen

ToDo:
- @Vincent: per gemaal de bovenstroomse GPG specificeren i.v.m. bepalen aan- afslagpeilen. Zie .\data\xlsx\kwk_gpg.csv met dank aan collega Johan Draaijer

In [8]:
#we lezen hier al de GPG's in, omdat we alleen gemalen toevoegen waarvan we een peil hebben
pumps_gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["gemalen"]),
                                 "pumps",
                                 column_mapping={"gecom_max_": "maximalecapaciteit"},
                                 keep_columns=["gpg_boven", "functie"])

# capaciteit van aanvoergemalen zetten we op 0 m3/s
pumps_gdf.loc[pumps_gdf["functie"] == "aanvoergemaal", "maximalecapaciteit"] = 0

gemalen_gdf = pumps_gdf.copy()
pumps_gdf["codegerelateerdobject"] = pumps_gdf["code"].copy()
pumps_gdf["code"] = [f"PMP_{code}" for code in pumps_gdf["code"]]
hydamo.gemalen.set_data(gemalen_gdf,
                        index_col="code",
                        check_columns=True,
                        check_geotype=True)
hydamo.gemalen.snap_to_branch(hydamo.branches, snap_method="overal", maxdist=5)
hydamo.gemalen.dropna(axis=0, inplace=True, subset=["branch_offset"])
hydamo.pumps.set_data(pumps_gdf,
                      index_col="code",
                      check_columns=True,
                      check_geotype=True)
hydamo.pumps.snap_to_branch(hydamo.branches, snap_method="overal", maxdist=1)
hydamo.pumps.dropna(axis=0, inplace=True, subset=["branch_offset"])



### Toevoegen bruggen
Per brug moet er een profiel worden toegevoegd

In [9]:
gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["bruggen"]),
                           "bridges",
                           attribute_filter={"KBRSOORT": [2, 3, 4]},
                           snap_to_branches=snap_to_branches,
                           column_mapping={"doorstro_2": "lengte",
                                           "hoogte_ond": "hoogteonderzijde"},
                          keep_columns = ["doorstroom"])

gdf = gdf.loc[gdf["hoogteonderzijde"] != 0]
gdf = gdf.loc[gdf["doorstroom"] != 0]
gdf.loc[gdf["lengte"] == 0, "lengte"] = 10

grouper = gdf.groupby("code")
data = {"code": [code for code, frame in grouper],
        "hoogteonderzijde": [frame["hoogteonderzijde"].values[0] for code,
                             frame in grouper],
        "lengte": [frame["lengte"].values[0] for code, frame in grouper],
        "geometry": [frame["geometry"].values[0] for code, frame in grouper]
        }
gdf = gpd.GeoDataFrame(data)

gdf["hoogtebovenzijde"] = gdf["hoogteonderzijde"] + 1

gdf["dwarsprofielcode"] = gdf["code"].apply(lambda x: f"PRO_{x}")
gdf["intreeverlies"] = 0.7
gdf["uittreeverlies"] = 0.7
gdf["ruwheidstypecode"] = 4
gdf["ruwheidswaarde"] = 70

gdf = gdf.loc[gdf["dwarsprofielcode"].isin(hydamo.parametrised_profiles.index)]

hydamo.bridges.set_data(gdf, index_col="code", check_columns=True, check_geotype=False)
hydamo.bridges.snap_to_branch(hydamo.branches, snap_method="overal", maxdist=1)
hydamo.bridges.dropna(axis=0, inplace=True, subset=["branch_offset"])

### Aanmaken sluizen (als duiker)

Sluizen worden toegevoegd als hoge duikers (10m). Ontbrekende waarden worden als volgt ingevuld:
- Aantal kokers = 1
- Lengte = 20m
- Breedte = breedte principe profiel onderliggende branch
- BOK = hoogte principe profiel onderliggende branch

In [10]:
sluices_gdf = hydrotools.read_file(
    beheerregister.joinpath(invoerbestanden["sluizen"]),
    "culverts",
    attribute_filter={"SOORT": ["schutsluis naar een zijde",
                                "overig",
                                "schutsluis naar twee zijden",
                                "uitwateringssluis/spuisluis"],
                     "STATUS": "gerealiseerd"},
    keep_columns=["AANTAL_SLU", "SOORT", "branch_id"],
    column_mapping={
        "KOLKWIJDTE": "breedteopening",
        "BOS_BENEDE": "hoogtebinnenonderkantbenedenstrooms",
        "BOS_BOVEN": "hoogtebinnenonderkantbovenstrooms",
        "KOLKLENGTE": "lengte"},
    snap_to_branches=snap_to_branches
    )

sluices_gdf.set_index("code", inplace=True)
sluices_gdf.replace({'0': None, 0: None}, inplace=True)
sluices_gdf.loc[sluices_gdf["lengte"].isnull(), "lengte"] = 20
sluices_gdf.loc[sluices_gdf["aantal_slu"].isnull(), "aantal_slu"] = 1
sluices_gdf["breedteopening"] = sluices_gdf.apply(
    (lambda x: principe_profielen_df.loc[x["branch_id"]][
        "bottomwidth"] if not x["breedteopening"] else x["breedteopening"]), axis=1)

sluices_gdf["hoogtebinnenonderkantbenedenstrooms"] = sluices_gdf.apply(
    (lambda x: principe_profielen_df.loc[x["branch_id"]][
        "bottomlevel"] if not x["hoogtebinnenonderkantbenedenstrooms"] else x[
            "hoogtebinnenonderkantbenedenstrooms"]), axis=1)

sluices_gdf["hoogtebinnenonderkantbovenstrooms"] = sluices_gdf.apply(
    (lambda x: principe_profielen_df.loc[x["branch_id"]][
        "bottomlevel"] if not x["hoogtebinnenonderkantbovenstrooms"] else x[
            "hoogtebinnenonderkantbovenstrooms"]), axis=1)

sluices_gdf.loc[:, "vormcode"] = "rechthoekig"
sluices_gdf.loc[:, "hoogteopening"] = 10
sluices_gdf["allowedflowdir"] = sluices_gdf.apply((lambda x: richting_mapping[x["soort"]]), axis=1)
sluices_gdf.reset_index(inplace=True)

sluices_gdf.loc[(sluices_gdf["code"] == "KSL011"), "breedteopening"] = 120

### Toevoegen duikers en siphons
duikers en sifons worden samen met de sluizen als cuverts aan het HyDAMO object toegekend

In [ ]:
culvert_groups = kwk_groups_df[kwk_groups_df["object_laag"] == "culverts"]["group_code"]

culverts_gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["duikers"]),
                                    "culverts",
                                    snap_to_branches=snap_to_branches,
                                    attribute_filter={"SOORT": "tussenduiker"},
                                    column_mapping={"hoogte_a1": "hoogteopening",
                                                    "breedte": "breedteopening",
                                                    "bok_benede": "hoogtebinnenonderkantbenedenstrooms",
                                                    "bok_boven": "hoogtebinnenonderkantbovenstrooms",
                                                    "vorm": "vormcode"})

# aanmaken culverts die niet op branch liggen, maar wel horen bij een groep
grouped_culverts_gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["duikers"]),
                                    "culverts",
                                    attribute_filter={"SOORT": "tussenduiker"},
                                    column_mapping={"hoogte_a1": "hoogteopening",
                                                    "breedte": "breedteopening",
                                                    "bok_benede": "hoogtebinnenonderkantbenedenstrooms",
                                                    "bok_boven": "hoogtebinnenonderkantbovenstrooms",
                                                    "vorm": "vormcode"})

grouped_culverts_gdf = grouped_culverts_gdf.loc[
    grouped_culverts_gdf["code"].isin(culvert_groups.index)
    ]

grouped_culverts_gdf = grouped_culverts_gdf.loc[
    ~grouped_culverts_gdf["code"].isin(culverts_gdf["code"])
    ]

siphons_gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["sifons"]),
                                   "culverts",
                                   snap_to_branches=snap_to_branches,
                                   column_mapping={"hoogte": "hoogteopening",
                                                   "breedte": "breedteopening",
                                                   "bok_benede":"hoogtebinnenonderkantbenedenstrooms",
                                                   "bok_boven":"hoogtebinnenonderkantbovenstrooms",
                                                   "vorm":"vormcode"})
"opruimen ten onrechte behouden sifons"
siphons_gdf = siphons_gdf.loc[~siphons_gdf["code"].isin(["KSY118",
                                                         "KSY102",
                                                         "KSY103",
                                                         "KSY114",
                                                         "KSY115",
                                                         "KSY126",
                                                         "KSY135",
                                                         "KSY140",
                                                         "KSY144",
                                                         "KSY176"])]

"toevoegen ten onrechte weggegooide sifons"
siphons_patch_gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["sifons"]),
                                   "culverts",
                                   column_mapping={"hoogte": "hoogteopening",
                                                   "breedte": "breedteopening",
                                                   "bok_benede":"hoogtebinnenonderkantbenedenstrooms",
                                                   "bok_boven":"hoogtebinnenonderkantbovenstrooms",
                                                   "vorm":"vormcode"})

siphons_patch_gdf = siphons_patch_gdf.loc[siphons_patch_gdf["code"].isin(["KSY101","KSY129"])]

gdf = gpd.GeoDataFrame(pd.concat([
    sluices_gdf, culverts_gdf, grouped_culverts_gdf, siphons_gdf, siphons_patch_gdf
    ], ignore_index=True))



gdf.loc[gdf['vormcode'].isnull(), 'vormcode'] = 'onbekend'
gdf.loc[:, 'vormcode'] = gdf.apply((lambda x: vorm_mapping[x['vormcode'].lower()]), axis=1)

gdf["intreeverlies"] = 0.7
gdf["uittreeverlies"] = 0.7
gdf["ruwheidswaarde"] = 70
gdf["ruwheidstypecode"] = 4

hydamo.culverts.set_data(gdf, index_col="code", check_columns=True, check_geotype=True)
hydamo.culverts.snap_to_branch(hydamo.branches, snap_method="ends", maxdist=25)
hydamo.culverts.dropna(axis=0, inplace=True, subset=["branch_offset"])

hydamo.culverts.loc[hydamo.culverts["lengte"].isna(), "lengte"] = 20
hydamo.culverts.loc[hydamo.culverts["allowedflowdir"].isna(), "allowedflowdir"] = "both"

# oplossen fouten in data, duiker op bodem waterloop leggen als 0
hydamo.culverts['hoogtebinnenonderkantbenedenstrooms'] = hydamo.culverts.apply(
    (lambda x: x['hoogtebinnenonderkantbenedenstrooms']
     if x['hoogtebinnenonderkantbenedenstrooms'] != 0
     else principe_profielen_df.loc[x["branch_id"]]['bottomlevel']),
    axis=1
    )

hydamo.culverts['hoogtebinnenonderkantbovenstrooms'] = hydamo.culverts.apply(
    (lambda x: x['hoogtebinnenonderkantbovenstrooms']
     if x['hoogtebinnenonderkantbovenstrooms'] != 0
     else principe_profielen_df.loc[x["branch_id"]]['bottomlevel']),
    axis=1
    )

### Toevoegen stuwen

In [ ]:
stuwen_ignore = ["KST0338"]

gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["stuwen"]),
                           "weirs",
                           attribute_filter={"kwkstatu": 300,
                                            "kstregel": [1, 2, 3, 4]},
                           column_mapping={"kstident": "code",
                                           "kstsoort": "soortstuwcode",
                                           "kstdsbre": "laagstedoorstroombreedte",
                                           "kstmikho": "laagstedoorstroomhoogte",
                                           "kstregel": "soortregelbaarheidcode"},
                          keep_columns=["KSTMIKHO", "KSTMAXKR", "GPG_BOVEN"])

gdf = gdf.loc[~gdf["code"].isin(stuwen_ignore)]

gdf["afvoercoefficient"] = 1

hydamo.weirs.set_data(gdf, index_col="code", check_columns=True, check_geotype=True)
hydamo.weirs.snap_to_branch(hydamo.branches, snap_method="overal", maxdist=0.5)
hydamo.weirs.dropna(axis=0, inplace=True, subset=["branch_offset"])

#fouten wegwerken uit data
hydamo.weirs["laagstedoorstroombreedte"] = hydamo.weirs["laagstedoorstroombreedte"].apply(lambda x: 0.1 if x == 0 else x)
hydamo.weirs["laagstedoorstroomhoogte"] = hydamo.weirs.apply(
    (lambda x: principe_profielen_df.loc[x["branch_id"]][
        "bottomlevel"] + 0.1 if x["laagstedoorstroomhoogte"] == 0 else x["laagstedoorstroomhoogte"]), axis = 1)

### Toevoegen inlaten als orifice

In [ ]:
inlaten_gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["inlaten"]),
                                   "orifices",
                                   snap_to_branches=snap_to_branches,
                                   column_mapping={"KWKKERHG": "laagstedoorstroomhoogte",
                                                  "kwkident": "code"})

hydamo.orifices.set_data(inlaten_gdf, index_col="code", check_columns=False, check_geotype=True)
hydamo.orifices.snap_to_branch(hydamo.branches, snap_method="overal", maxdist=0.5)
hydamo.orifices.dropna(axis=0, inplace=True, subset=["branch_offset"])

hydamo.orifices["laagstedoorstroomhoogte"] = hydamo.orifices.apply(
    (lambda x: principe_profielen_df.loc[x["branch_id"]][
        "bottomlevel"] + 0.1 if x["laagstedoorstroomhoogte"] == 0 else x["laagstedoorstroomhoogte"]), axis = 1)
hydamo.orifices['laagstedoorstroombreedte'] = 0.3
hydamo.orifices['schuifhoogte'] = hydamo.orifices["laagstedoorstroomhoogte"]
hydamo.orifices['afvoercoefficient'] = 0.7

### Kunstwerken opruimen

Opruimen in de volgende volgorde.
- Als er op een tak een brug binnen een rekenpuntafstand van een duiker ligt, dan verwijderen we die
- Als er op een tak een duiker binnen een rekenpuntafstand van een gemaal ligt, dan verwijderen we die

In [ ]:
gdf = hydamo.bridges.loc[
        hydamo.bridges.apply(
        hydrotools.filter_to_other_object,
        args=(hydamo.culverts, rekenpunt_afstand),
        axis=1)
    ]
hydamo.bridges.set_data(gdf, index_col="code", check_columns=True, check_geotype=False)

gdf = hydamo.culverts.loc[
        hydamo.culverts.apply(
        hydrotools.filter_to_other_object,
        args=(hydamo.gemalen, rekenpunt_afstand),
        axis=1)
    ]
hydamo.culverts.set_data(gdf, index_col="code", check_columns=True, check_geotype=False)

### Toevoegen afsluitmiddelen

In [ ]:
gdf = pd.DataFrame({"code": [],
                    "soortafsluitmiddelcode": [],
                    "codegerelateerdobject": []})

hydamo.afsluitmiddel.set_data(gdf, index_col="code")

### Toevoegen sturing

Voeg sturing toe uit peilgebieden-bestand. Alleen voor stuwen van typen 2,3,4 en gemalen, mits de gpg_boven kolom gevuld is met waarden die in de peilgebieden zijn gegeven.

In [ ]:
kst_gestuurd = hydamo.weirs.loc[
    hydamo.weirs['soortregelbaarheidcode'].isin([2,3,4])
    ]
kst_gestuurd = kst_gestuurd["gpg_boven"]
kgm_gestuurd = hydamo.gemalen["gpg_boven"]
sturing_df = pd.DataFrame(pd.concat([kgm_gestuurd, kst_gestuurd], axis = 0))
sturing_df.reset_index(inplace=True)
sturing_df.rename({"code": "codegerelateerdobject"}, axis=1, inplace=True)
sturing_df["code"] = sturing_df.apply(
    (lambda x: f'{x["codegerelateerdobject"]}_{x["gpg_boven"]}'),
    axis=1
    )

gpg_gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["peilgebieden"]),
                                 "sturing",
                                 column_mapping={"GPGIDENT": "code",
                                                 "OPVAFWZP": "streefwaarde"
                                                 })

sturing_df.loc[~sturing_df["gpg_boven"].isin(gpg_gdf["code"]), "gpg_boven"] = None

sturing_df.loc[:, "streefwaarde"] = sturing_df["gpg_boven"].apply(
    lambda x: gpg_gdf.set_index("code").loc[x]["streefwaarde"] if x is not None else None
    )

# bij deze gemalen is geen (bestaande) gpg opgegeven
sturing_df.loc[sturing_df["codegerelateerdobject"].isin(["HEA067",
                                                         "WSF218",
                                                         "WSF219",
                                                         "KGM098"]),
               "streefwaarde"] = 10

sturing_df.loc[:, "bovenmarge"] = sturing_df.apply(
    (lambda x: hydamo.weirs.loc[x["codegerelateerdobject"]]["kstmaxkr"] if x["codegerelateerdobject"] in hydamo.weirs.index
    else x["streefwaarde"] + 0.05), axis=1)
sturing_df.loc[:, "ondermarge"] = sturing_df.apply(
    (lambda x: hydamo.weirs.loc[x["codegerelateerdobject"]]["laagstedoorstroomhoogte"] if x["codegerelateerdobject"] in hydamo.weirs.index
    else x["streefwaarde"] - 0.05), axis=1)
sturing_df.loc[:, "doelvariabelecode"] = 1

hydamo.sturing.set_data(sturing_df, index_col="code")

# hier ontbreekt ook een gpg
hydamo.sturing.loc["KST0421_GPGKST1075", "streefwaarde"] = -0.75

### Exporteren

Om het resultaat te beoordelen exporteren we alles naar shape-files. We slaan het hydamo object ook op als "pickle", zodat we bovenstaande stappen niet elke keer hoeven te herhalen

In [ ]:
hydrotools.export_shapes(hydamo, path=Path(r"./hydamo_shp/boezemmodel_v4"))
hydrotools.save_model(hydamo, file_name=Path(r"./hydamo_model/boezemmodel_v4.pickle"))


## Converteren naar DFM

### Aanmaken dfm-klasse

In [ ]:
dfmmodel = DFlowFMModel()

### Inlezen kunstwerken

In [ ]:
#hydamo.gemalen = hydamo.gemalen.loc[hydamo.gemalen.index.isin(hydamo.sturing["codegerelateerdobject"])]
#hydamo.pumps = hydamo.pumps.loc[hydamo.pumps.index.isin(hydamo.sturing["codegerelateerdobject"])]
dfmmodel.structures.io.weirs_from_hydamo(hydamo.weirs,
                                         sturing=hydamo.sturing,
                                         yz_profiles=hydamo.crosssections,
                                         parametrised_profiles=hydamo.parametrised_profiles)

dfmmodel.structures.io.culverts_from_hydamo(hydamo.culverts,
                                            hydamo.afsluitmiddel)

dfmmodel.structures.io.orifices_from_hydamo(hydamo.orifices)

dfmmodel.structures.io.bridges_from_hydamo(hydamo.bridges,
                                           yz_profiles=hydamo.crosssections,
                                           parametrised_profiles=hydamo.parametrised_profiles)

dfmmodel.structures.io.orifices_from_hydamo(hydamo.orifices)

dfmmodel.structures.io.pumps_from_hydamo(pompen=hydamo.pumps,
                                         sturing=hydamo.sturing,
                                         gemalen=hydamo.gemalen)

for key, value in dfmmodel.structures.culverts.items():
    value["allowedFlowDir"] = hydamo.culverts.loc[key]["allowedflowdir"]

### aanmaken compound structures

In [ ]:
cmpnd_ids = [group for group, _ in kwk_groups_df.groupby("group_code")]
cmpnd_list = [df.index.to_list() for _, df in kwk_groups_df.groupby("group_code")]
dfmmodel.structures.io.compound_structures(cmpnd_ids, cmpnd_list)

### Aanmaken 1D netwerk
We maken een 1D model met een rekenpuntafstand van 250m

In [ ]:
dfmmodel.network.set_branches(hydamo.branches)
dfmmodel.network.generate_1dnetwork(one_d_mesh_distance=rekenpunt_afstand, seperate_structures=True)

### Toevoegen cross-sections

Toevoegen HyDAMO profielen

In [ ]:
dfmmodel.crosssections.io.from_hydamo(
    dwarsprofielen=hydamo.crosssections,
    parametrised=hydamo.parametrised_profiles,
    branches=hydamo.branches
)

# Toevoegen profielen voor Lauwersmeer uit bathymetrie

We halen profielen uit de GeoTiff met bathymetrie hoogten.

In [ ]:
bathymetrie_raster = bathymetrie.joinpath(bathymetriebestanden['hoogte_raster'])
bathymetrie_zones = bathymetrie.joinpath(bathymetriebestanden['zones'])

stats = ['min']
stats += ['percentile_{}'.format(perc) for perc in range(1, 5, 2)]
stats += ['percentile_{}'.format(perc) for perc in range(5, 20, 5)]
stats += ['percentile_{}'.format(perc) for perc in range(20, 80, 10)]
stats += ['percentile_{}'.format(perc) for perc in range(80, 100, 2)]
stats += ['max']

gdf = gpd.read_file(bathymetrie_zones)

with rasterio.open(bathymetrie_raster, "r") as src:
    profile = src.profile
    raster_data = src.read(1)
    affine = src.transform
    cell_area = profile['transform'][0]**2

raster_stats = zonal_stats(gdf,
                           raster_data,
                           affine=affine,
                           stats=stats,
                           nodata=profile["nodata"],
                           raster_out=True)

data = {}

for idx, (_, row) in enumerate(gdf.iterrows()):
    ident = row["ID"]
    branches = hydamo.branches.loc[
         hydamo.branches["geometry"].centroid.within(row["geometry"])
        ].index
    length = hydamo.branches.loc[branches]["geometry"].length.sum()
    data[ident] = {}
    for stat in stats:
        data[ident]["z"] = [raster_stats[idx][stat] for stat in stats]
        data[ident]['area'] = [
            np.sum(
                raster_stats[idx]['mini_raster_array'] <= raster_stats[idx][stat]
                ) * cell_area for stat in stats
            ]

    data[ident]['lenght'] = [item/length for item in data[ident]['area']]

    z = list(reversed(data[ident]["z"])) + data[ident]["z"]
    y = list(
        -(np.array(data[ident]["lenght"]) / 2)[::-1]
        ) + list(np.array(data[ident]["lenght"]) / 2)
    yz = np.array(list(zip(y, z)))
    mask = [True] + [not(item.all()) for item in np.equal(yz[1:], np.array(yz[:-1]))]
    yz = np.array([item for idx, item in enumerate(yz) if mask[idx]])

    definition = f"PRO_{ident}"
    dfmmodel.crosssections.add_yz_definition(yz=yz,
                                             thalweg=0,
                                             roughnesstype='Manning',
                                             roughnessvalue=0.035,
                                             name=definition)

    for branch in branches:
        chainage = hydamo.branches.loc[branch]["geometry"].length/2
        dfmmodel.crosssections.add_crosssection_location(branch,
                                                         chainage,
                                                         definition=definition)

Toevoegen principeprofielen op takken die nog niet over een profiel beschikken

In [ ]:
if len(dfmmodel.crosssections.get_branches_without_crosssection()) > 0:
    print("adding trapezium profiles on branches with missing crosssections.")
    dfmmodel = hydrotools.add_trapeziums(dfmmodel, principe_profielen_df)

### Toevoegen initiele condities

In [ ]:
dfmmodel.external_forcings.set_initial_waterdepth(6)
# gdf = gpd.GeoDataFrame.from_file(beheerregister.joinpath(invoerbestanden["peilgebieden"]))
# gdf = gdf.explode()
# gdf["INDEX"] = ""

#for index, df in gdf.groupby("GPGIDENT"):
#    for idx, (_, row) in enumerate(df.iterrows()):
#        if len(df) > 1:
#            gdf.loc[row.name, 'INDEX'] = f"{index}_{idx + 1:02d}"
#    else:
#        gdf.loc[row.name, 'INDEX'] = index

#gdf.set_index('INDEX', inplace=True)
#gdf["waterlevel"] = gdf["OPVAFWWP"] + 0.2
#dfmmodel.external_forcings.initial_waterlevel_polygons = gdf

### Toevoegen randvoorwaarden

In [ ]:
rvw_df = pd.read_excel(excelbestanden.joinpath(modelbestanden["randvoorwaarden"]))

for _,row in rvw_df.iterrows():
    branch_id = row["BRANCH_ID"]
    pt = hydamo.branches.loc[branch_id]["geometry"].coords[int(row["COORD"])]
    series = pd.Series(data=[0.0, 0.0],
                       index=[pd.Timestamp("2000-01-01"),
                              pd.Timestamp("2100-01-01")]
                       )
    #Bij Cleveringsluizen setten we de randvoorwaarde op peil lauwersmeer
    if branch_id == "OAF005301":
        series = series - 0.93
    dfmmodel.external_forcings.add_boundary_condition(name=f"BND_{branch_id}",
                                                      pt=pt,
                                                      bctype=row["TYPE"],
                                                      series=series)

### Wegschrijven model

Note: in boundaries.bc gaat in versi 1.1.3 van delft3dfmpy iets fout:
2000-01-01 00:00:00 wordt 2000-01-01 :00:00. Hier crashed de interface van D-Hydro op

In [ ]:
dfmmodel.mdu_parameters["refdate"] = 20000101
dfmmodel.mdu_parameters["tstart"] = 0.0 * 3600
dfmmodel.mdu_parameters["tstop"] = 10 * 24 * 3600
dfmmodel.mdu_parameters["hisinterval"] = "600. 0. 0."
dfmmodel.mdu_parameters["mapinterval"] = "600. 0. 0."
dfmmodel.mdu_parameters["wrirst_bnd"] = 0
dfmmodel.mdu_parameters["cflmax"] = 0.7
dfmmodel.mdu_parameters["outputdir"] = "output_initialisatie"

dfmmodel.dimr_path = dimr_path
fm_writer = DFlowFMWriter(dfmmodel, output_dir=r"modellen\boezemmodel", name="boezemmodel")

fm_writer.objects_to_ldb()
fm_writer.write_all()

if Path(r"modellen\boezemmodel\fm\boezemmodel.mdu").exists():
    print("Model is weggeschreven")
else:
    print("Er is geen model geschreven. Waarschijnlijk is iets fout gegaan")

## Importeren in de D-Hydro suite

Open het model nu in D-Hydro:
1. Open een "Empty Project"
2. In de "Home" ribbon, ga naar "Import" en selecteer "Flow Flexible Mesh Model"
3. Selecteer het bestand "boezemmodel.mdu" in ".\dfm_model\fm\boezemmodel.mdu"
4. Wacht tot het model is geimporteerd.....

Het geimporteerde model is nu zichtbaar in de D-Hydro suite
<img src="png/boezemmodel.png"/>